## Imports

In [1]:
import sys
import os
import numpy as np
import numpy.lib.recfunctions as nprf
from sklearn import linear_model
import matplotlib as mpl

from importlib import reload
import pandas as pd

mpl.use('Agg')

mpl.rcParams['figure.figsize'] = (7,7)
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
mpl.rcParams.update({'font.size':22})

% matplotlib inline

import matplotlib.pyplot as plt

In [3]:
import tools.matt_tools as matt

reload(matt)

<module 'tools.matt_tools' from '/home/mho1/halo_cnn/notebooks/tools/matt_tools.py'>

In [4]:
wdir = '/home/mho1/scratch/halo_cnn'

# Load

In [5]:
data_file = os.path.join(wdir,
                         'data_raw',
                         'MDPL2_Rockstar_z=0.117_Macc=1e11.csv')

In [ ]:
dat_MD = pd.read_csv(data_file)

In [ ]:
dat_MD.head()

# Scratch

In [2]:
from collections import OrderedDict

In [4]:
par = OrderedDict()

In [17]:
par1 = OrderedDict([
    ('wdir', '/home/mho1/scratch/halo_cnn/')
])
par2 = OrderedDict([
    ('model_name', 'halo_cnn1d_r')
])

In [13]:
for key in par.keys():
    print(key + ' : ' + str(par[key]))

wdir : /home/mho1/scratch/halo_cnn/
model_name : halo_cnn1d_r


In [18]:
par1.update(par2)

In [23]:
par1

OrderedDict([('wdir', '/home/mho1/scratch/halo_cnn/'),
             ('model_name', 'halo_cnn1d_r')])

In [4]:
from bs4 import BeautifulSoup

In [6]:
xml = '''<uws:job xmlns:uws="http://www.ivoa.net/xml/UWS/v1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="1.1">
  <uws:jobId>1528838918683310284</uws:jobId>
  <uws:runId xsi:nil="true"/>
  <uws:ownerId>maho3</uws:ownerId>
  <uws:phase>PENDING</uws:phase>
  <uws:quote xsi:nil="true"/>
  <uws:creationTime>2018-06-12T23:28:38+02:00</uws:creationTime>
  <uws:startTime xsi:nil="true"/>
  <uws:endTime xsi:nil="true"/>
  <uws:executionDuration>0</uws:executionDuration>
  <uws:destruction xsi:nil="true"/>
  <uws:parameters/>
  <uws:results/>
</uws:job>
'''

In [11]:
soup = BeautifulSoup(xml,'xml')

In [15]:
soup.find('uws:jobId').getText()

'1528838918683310284'